In [1]:
#coding=utf-8
import zmq
import sys
import numpy as np
import pandas as pd
from time import sleep
import DQN_Agent_pb2 


In [ ]:
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('ipc:///tmp/mdcumd')
socket.setsockopt_string(zmq.SUBSCRIBE,'')
pd_md_data = pd.DataFrame()
price_raw_data = []
volume_raw_data = []
timestamp_raw_data = []

i = 0
AGENT_ACTION = DQN_Agent_pb2.Agent_Info()
MD =DQN_Agent_pb2.MD_Info()

In [ ]:
while(True):
    print(i)
    i += 1
    r_msg = socket.recv()
    MD.ParseFromString(r_msg)
    
    timestamp_raw_data.append(MD.MD_Timestamp)
    volume_raw_data.append(MD.volume)
    price_raw_data.append(MD.last_price)
    
    if i % 200 == 0:
        pd_md_data['TradeTime'] = timestamp_raw_data
        pd_md_data['last_price'] = price_raw_data
        pd_md_data['volume'] = volume_raw_data
        pd_md_data.set_index('TradeTime')
        pd_md_data.to_csv(MD.MD_Instrument)

0
